In [1]:
import mlflow
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from mlflow.tracking import MlflowClient
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import imutils
from mlflow import MlflowClient

In [2]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'C:\Users\Clsm\Documents\GitHub\telerehab_ai\credentials\serviceAccount.json'

In [3]:
MLFLOW_TRACKING_URI = 'http://34.173.115.10:8888'

In [4]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment("epm_test")

<Experiment: artifact_location='gs://aiteam-mlflow-storage-bucket/models/22', creation_time=1679988551551, experiment_id='22', last_update_time=1679988551551, lifecycle_stage='active', name='epm_test', tags={}>

In [2]:
#mlflow.set_tracking_uri("http://34.173.115.10:8888")
#mlflow.set_tracking_uri("sqlite:///mlflow.db")

## Loading Datasets

In [5]:
train_dir = r'C:\Users\Clsm\fruit_dataset\train'
test_dir = r'C:\Users\Clsm\fruit_dataset\test'

In [6]:
train_datagen = ImageDataGenerator(rescale= 1./255)
test_datagen = ImageDataGenerator(rescale= 1./255)

In [7]:
batch_size = 16
train_set = train_datagen.flow_from_directory(train_dir,
                                            target_size= (100,100),
                                            batch_size = batch_size,
                                            class_mode = 'categorical')
test_set = test_datagen.flow_from_directory(test_dir,
                                            target_size = (100,100),
                                            batch_size = batch_size,
                                            class_mode = 'categorical')

Found 2301 images belonging to 10 classes.
Found 1025 images belonging to 10 classes.


## Model Creating

In [8]:
IMAGE_SIZE = [100,100]
vgg = VGG19(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
#do not train the pre-trained layers of VGG-19
for layer in vgg.layers:
    layer.trainable = False
x = Flatten()(vgg.output)
#adding output layer.Softmax classifier is used as it is multi-class classification
prediction = Dense(10, activation='softmax')(x)

model = Model(inputs=vgg.input, outputs=prediction)

In [9]:
model.compile(
  loss='categorical_crossentropy',
  optimizer="adam",
  metrics=['accuracy']
)

## Model train and save in ML flow

In [11]:
import mlflow.keras
mlflow.sklearn.autolog()
with mlflow.start_run() as run:
    mlflow.set_tag("developer", "epm")
    mlflow.set_tag("version", "prerelease")

    mlflow.log_param('dataset' , 'fruit_classification')
    mlflow.log_param('train_amount', '2301images')
    mlflow.log_param('test_amount', '1025images')
    mlflow.keras.log_model(model, artifact_path="model")
    history = model.fit(train_set, batch_size = batch_size, epochs= 1)
    mlflow.log_metric('train_loss', history.history['loss'][-1])
    mlflow.log_metric('train_acc', history.history['accuracy'][-1])

2023/03/29 09:51:56 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.


INFO:tensorflow:Assets written to: C:\Users\Clsm\AppData\Local\Temp\tmpv4adjev4\model\data\model\assets
144/144 [==============================] - 269s 2s/step - loss: 0.9541 - accuracy: 0.7058


## Creating Model Version

In [12]:
run_id = run.info.run_id
name = "test"
client = MlflowClient(tracking_uri= MLFLOW_TRACKING_URI)
model_uri = "runs:/{}/model".format(run_id)
mv = client.create_model_version(name, model_uri, run_id)
print("model version {} created".format(mv.version))

2023/03/29 09:59:51 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: test, version 6


model version 6 created


In [13]:
client.delete_model_version(name, version='2')

## Creating another version

In [14]:
with mlflow.start_run() as run1:
    hist = model.fit(train_set, epochs=2, batch_size = 32)
    mlflow.sklearn.log_model(hist, artifact_path="model")

model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri,name=name)

Epoch 1/2
144/144 [==============================] - 245s 2s/step - loss: 0.9351 - accuracy: 0.7093
Epoch 2/2
144/144 [==============================] - 240s 2s/step - loss: 0.7242 - accuracy: 0.7705


2023/03/28 14:41:10 WARNING mlflow.tensorflow: Failed to infer model signature: could not sample data to infer model signature: '>=' not supported between instances of 'slice' and 'int'


INFO:tensorflow:Assets written to: C:\Users\Clsm\AppData\Local\Temp\tmpm2_hktzh\model\data\model\assets
INFO:tensorflow:Assets written to: ram://7041d754-6369-4f72-a6c7-cff4f274bd58/assets


2023/03/28 14:41:49 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2023/03/28 14:41:59 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\Clsm\AppData\Local\Temp\tmpw96vm2dc\model\model.pkl, flavor: sklearn), fall back to return ['scikit-learn==1.2.0', 'cloudpickle==2.2.0']. Set logging level to DEBUG to see the full traceback.
Registered model 'test' already exists. Creating a new version of this model...
2023/03/28 14:42:08 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: test, version 3
Created version '3' of model 'test'.


<ModelVersion: creation_timestamp=1679991132722, current_stage='None', description='', last_updated_timestamp=1679991132722, name='test', run_id='1b25ddd9e6bf498593b093e230fb3e95', run_link='', source='gs://aiteam-mlflow-storage-bucket/models/22/1b25ddd9e6bf498593b093e230fb3e95/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='3'>

In [13]:
client = MlflowClient(tracking_uri= MLFLOW_TRACKING_URI)
name = 'test'

In [14]:
model_versions = client.get_latest_versions(name= name)
for version in model_versions:
    print(f"version:{version.version}, stage:{version.current_stage}")

version:1, stage:Staging
version:3, stage:Production
version:6, stage:None


In [15]:
new_stage = 'Archived'
model_version = 6
client.transition_model_version_stage(
    name = name,
    version = model_version,
    stage = new_stage,
    archive_existing_versions=False
)

<ModelVersion: creation_timestamp=1680060596735, current_stage='Archived', description='', last_updated_timestamp=1680060644764, name='test', run_id='b4f5b0cd1b144c26b892cc3859f51d16', run_link='', source='runs:/b4f5b0cd1b144c26b892cc3859f51d16/model', status='READY', status_message='', tags={}, user_id='', version='6'>

In [16]:
artifact = []
def fetch_logged_data(run_id):
    client = MlflowClient()
    data = client.get_run(run_id).data
    tags = {k: v for k, v in data.tags.items() if not k.startswith("mlflow.")}
    #artifacts = [f.path for f in client.list_artifacts(run_id,'fruits-model_test1')]
    for f in client.list_artifacts(run_id,'model'):
        artifact.append(f.path)
    return data.params, data.metrics, tags, artifact
params, metrics, tags, artifact = fetch_logged_data(run_id=run_id)

In [17]:
from pprint import pprint
pprint(artifact)

['model/MLmodel',
 'model/conda.yaml',
 'model/data',
 'model/python_env.yaml',
 'model/requirements.txt']


## Model loading and evaluation

In [18]:
from mlflow.store.artifact.runs_artifact_repo import RunsArtifactRepository
runs_uri = f"runs:/{run.info.run_id}/model"
model_src = RunsArtifactRepository.get_underlying_uri(runs_uri)
model_src

'gs://aiteam-mlflow-storage-bucket/models/22/b4f5b0cd1b144c26b892cc3859f51d16/artifacts/model'

In [23]:
def test_model(name, stage, test_gen,):
    model_path = f"models:/{name}/{stage}"
    tf_model = mlflow.keras.load_model(model_path)

    results = tf_model.evaluate(test_gen , batch_size=batch_size)

    return {"test_loss": results[0],"test_acc" : results[1]}

In [24]:
%time test_model(name='test', stage="Archived",test_gen=test_set)

65/65 [==============================] - 122s 2s/step - loss: 1.9071 - accuracy: 0.4663
Wall time: 2min 34s


{'test_loss': 1.9071025848388672, 'test_acc': 0.46634146571159363}